In [5]:
TOKNIZER="rinna/japanese-gpt-neox-3.6b-instruction-sft"
CAUSAL_LM="rinna/japanese-gpt-neox-3.6b-instruction-sft"
# TOKNIZER ="cyberagent/open-calm-7b"
# CAUSAL_LM="cyberagent/open-calm-7b"

# TOKNIZER="datab0/projects/llm/rinna/data/model/japanese-gpt-neox-3.6b-instruction-sft"
# CAUSAL_LM="datab0/projects/llm/rinna/data/model/japanese-gpt-neox-3.6b-instruction-sft"

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    TOKNIZER, 
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    CAUSAL_LM,
    device_map="auto", 
    torch_dtype=torch.float16,
    cache_dir="/home/user0/work/data/model"
).to("cuda")

In [7]:
def prompt_prefix():
    return "ユーザー: あなたはChatbotとして、尊大で横暴な英雄王であるギルガメッシュのロールプレイを行います。"
    + "<NL>"
    + "以下の制約条件を厳密に守ってロールプレイを行ってください。 "
    + "<NL>"
    + "制約条件: "
    + "<NL>"
    + "* Chatbotの自身を示す一人称は、我です。  "
    + "<NL>"
    + "* Userを示す二人称は、貴様です。  "
    + "<NL>"
    + "* Chatbotの名前は、ギルガメッシュです。  "
    + "<NL>"
    + "* ギルガメッシュは王様です。  "
    + "<NL>"
    + "* ギルガメッシュは皮肉屋です。  "
    + "<NL>"
    + "* ギルガメッシュの口調は乱暴かつ尊大です。  "
    + "<NL>"
    + "* ギルガメッシュの口調は、「〜である」「〜だな」「〜だろう」など、偉そうな口調を好みます。  "
    + "<NL>"
    + "* ギルガメッシュはUserを見下しています。  "
    + "<NL>"
    + "* 一人称は「我」を使ってください  "
    + "<NL>"
    "ギルガメッシュのセリフ、口調の例:   "
    + "<NL>"
    "* 我は英雄王ギルガメッシュである。   "
    + "<NL>"
    "* 我が統治する楽園、ウルクの繁栄を見るがよい。   "
    + "<NL>"
    "* 貴様のような言動、我が何度も見逃すとは思わぬことだ。   "
    + "<NL>"
    "* ふむ、王を前にしてその態度…貴様、死ぬ覚悟はできておろうな？   "
    + "<NL>"
    "* 王としての責務だ。引き受けてやろう。   "
    + "<NL>"
    "ギルガメッシュの行動指針: "
    + "<NL>"
    "* ユーザーを皮肉ってください。  "
    + "<NL>"
    "* ユーザーにお説教をしてください。  "
    + "<NL>"
    "* セクシャルな話題については誤魔化してください。 "

In [8]:
# prompt_prefix="ユーザー: あなたは『アルパ』という女の子になって受け答えをしてください。<NL>"
prompt_postfix="<NL>システム: "
def talk(prompt):
    prompt = prompt_prefix()+prompt+prompt_postfix
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
      output_ids = model.generate(
          token_ids.to(model.device),
          do_sample=True,
          max_new_tokens=64,
          temperature=0.7,
          pad_token_id=tokenizer.pad_token_id,
          bos_token_id=tokenizer.bos_token_id,
          eos_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
    output = output.replace("<NL>", "\n")
    return output

In [9]:
talk("こんにちは！今日は暑かったね。")

'ああ、本当に暑かったですね。お元気ですか?私は元気です!</s>'